In [2]:
from utils.chat import OllamaChat
from utils.prompts import get_persona
from company_about_summarizer import summarize_company_about
from utils.example_company.company_about import company_data

# Summarizing the Company Data

In [3]:
company_data_summarized=summarize_company_about(company_data)


In [4]:
system_prompt = get_persona("general_sales_agent",company_data=company_data_summarized)

In [5]:
chat = OllamaChat(
        model='llama3.1',  # Replace with your desired model
        system_prompt=system_prompt,
        n_ctx=4096,
        temperature=0.7
    )

In [10]:
result=chat.chat(user_prompt="ummm yeah sure I am running an ecommerce business for cosmetics how can you help me ?")
# chat._save_conversation()

In [11]:
print(result)

That's fantastic! E-commerce and cosmetics are a great combination.

At TechCare AI, we've worked with several e-commerce businesses in the beauty and cosmetics space to improve their customer service experience. Our AI-powered solutions can help you automate responses to common questions, freeing up your team to focus on more complex issues.

For example, our ServiceFlow AI product can route tickets to the most suitable agents based on the issue, ensuring that customers get timely assistance. We've also seen significant improvements in response times and customer satisfaction with our clients who have implemented this solution.

Additionally, our ChatGenius Pro chatbot can help you provide 24/7 support to your customers, reducing the load on your human representatives during peak hours. And with our InsightHub Analytics, you'll get valuable insights into customer behavior and preferences, allowing you to make data-driven decisions for future improvements.

We've had success stories wi

In [27]:
chat._load_conversation()

In [42]:
result=chatbot.chat(message="What Services Do you provide ?")